In [6]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm


In [7]:
df = pd.read_csv("../data/kickstarter_projects.csv")

df = df[df['State'].isin(['Successful', 'Failed'])].copy()

df['Success'] = (df['State'] == 'Successful').astype(int)

display(df['Success'].value_counts())

Success
0    197611
1    133851
Name: count, dtype: int64

In [8]:
country_stats = df.groupby('Country')['Success'].agg(['count', 'mean']).sort_values('count', ascending=False)
country_stats.rename(columns={'count': 'n_projects', 'mean': 'success_rate'}, inplace=True)

display(country_stats.head(20))


,n_projects,success_rate
Country,,
United States,261358,0.418196
United Kingdom,29453,0.409704
Canada,12370,0.334196
Australia,6616,0.303809
Germany,3436,0.272701
France,2520,0.360317
Netherlands,2411,0.255910
Italy,2369,0.185310
Spain,1873,0.262680


In [9]:
min_projects = 400
country_counts = df['Country'].value_counts()

major_countries = country_counts[country_counts >= min_projects].index

df['Country_simplified'] = df['Country'].where(df['Country'].isin(major_countries), 'Other')

display(df['Country_simplified'].value_counts())


Country_simplified
United States     261358
United Kingdom     29453
Canada             12370
Australia           6616
Germany             3436
France              2520
Netherlands         2411
Italy               2369
Spain               1873
Sweden              1509
Mexico              1411
New Zealand         1274
Denmark              926
Ireland              683
Switzerland          652
Norway               582
Belgium              523
Austria              485
Hong Kong            477
Singapore            454
Other                 80
Name: count, dtype: int64

In [10]:
# one-hot code
country_dummies = pd.get_dummies(df['Country_simplified'], prefix='Country', drop_first=True)
country_dummies = country_dummies.astype(int)

display(country_dummies.head())
display(df['Success'].dtype)
display(country_dummies.dtypes)

,Country_Austria,Country_Belgium,Country_Canada,Country_Denmark,Country_France,Country_Germany,Country_Hong Kong,Country_Ireland,Country_Italy,Country_Mexico,Country_Netherlands,Country_New Zealand,Country_Norway,Country_Other,Country_Singapore,Country_Spain,Country_Sweden,Country_Switzerland,Country_United Kingdom,Country_United States
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


dtype('int32')

Country_Austria           int32
Country_Belgium           int32
Country_Canada            int32
Country_Denmark           int32
Country_France            int32
Country_Germany           int32
Country_Hong Kong         int32
Country_Ireland           int32
Country_Italy             int32
Country_Mexico            int32
Country_Netherlands       int32
Country_New Zealand       int32
Country_Norway            int32
Country_Other             int32
Country_Singapore         int32
Country_Spain             int32
Country_Sweden            int32
Country_Switzerland       int32
Country_United Kingdom    int32
Country_United States     int32
dtype: object

In [11]:
# x: dummies for all countries
X_country = country_dummies

# add constant
X_country = sm.add_constant(X_country)

# Y：Success
y = df['Success']

# fit the model
model_country = sm.Logit(y, X_country).fit()

display(model_country.summary())
  

Optimization terminated successfully.
         Current function value: 0.671035
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Success   No. Observations:               331462
Model:                          Logit   Df Residuals:                   331441
Method:                           MLE   Df Model:                           20
Date:                Wed, 03 Dec 2025   Pseudo R-squ.:                0.005182
Time:                        18:47:55   Log-Likelihood:            -2.2242e+05
converged:                       True   LL-Null:                   -2.2358e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.8292      0.027    -31.019      0.000      -0.882      -0.777
Country_Austria           -0.4328      0.113     -3.840      0.000      -0.654      -0.212
Country_Belgium           -0.0631      0.100     -0.631      0.528      -0.259       0.133
Country_Canada             0.1400      0.033      4.263      0.000       0.076       0.204
Country_Denmark            0.3767      0.073      5.195      0.000       0.235       0.519
Country_France             0.2552      0.049      5.171      0.000       0.158       0.352
Country_Germany           -0.1517      0.047     -3.248      0.001      -0.243      -0.060
Country_Hong Kong          0.6400      0.096      6.681      0.000       0.452       0.828
Country_Ireland           -0.0035      0.087     -0.040      0.968      -0.175       0.168
Country_Italy             -0.6516      0.059    -10.996      0.000      -0.768      -0.535
Country_Mexico            -0.1120      0.065     -1.723      0.085      -0.239       0.015
Country_Netherlands       -0.2381      0.054     -4.427      0.000      -0.344      -0.133
Country_New Zealand        0.2174      0.064      3.372      0.001       0.091       0.344
Country_Norway            -0.1234      0.096     -1.282      0.200      -0.312       0.065
Country_Other              0.0983      0.240      0.409      0.682      -0.372       0.569
Country_Singapore          0.3906      0.100      3.915      0.000       0.195       0.586
Country_Spain             -0.2029      0.059     -3.443      0.001      -0.318      -0.087
Country_Sweden             0.1539      0.061      2.538      0.011       0.035       0.273
Country_Switzerland       -0.0817      0.091     -0.902      0.367      -0.259       0.096
Country_United Kingdom     0.4640      0.029     15.870      0.000       0.407       0.521
Country_United States      0.4990      0.027     18.466      0.000       0.446       0.552
==========================================================================================
"""

In [13]:
# multiple variable regression
# transform to datetime
df['Launched_dt'] = pd.to_datetime(df['Launched'])
df['Deadline_dt'] = pd.to_datetime(df['Deadline'])

df['Launched_year'] = df['Launched_dt'].dt.year
df['duration_days'] = (df['Deadline_dt'] - df['Launched_dt']).dt.days

# log the goals
df['log_goal'] = np.log1p(df['Goal'])
df['log_backers'] = np.log1p(df['Backers'])

num_vars = df[['log_goal', 'log_backers', 'duration_days']]

# category varible
# Category dummy
cat_dummies = pd.get_dummies(df['Category'], prefix='Cat',drop_first=True)

# Subcategory dummy
subcat_dummies = pd.get_dummies(df['Subcategory'], prefix='Subcat', drop_first=True)

cat_dummies = cat_dummies.astype(int)
subcat_dummies = subcat_dummies.astype(int)


In [15]:
X_big = pd.concat([country_dummies,cat_dummies, num_vars], axis=1)

X_big = sm.add_constant(X_big)

y = df['Success']

model_big = sm.Logit(y, X_big).fit()

display(model_big.summary())

Optimization terminated successfully.
         Current function value: 0.170297
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Success   No. Observations:               331462
Model:                          Logit   Df Residuals:                   331424
Method:                           MLE   Df Model:                           37
Date:                Wed, 03 Dec 2025   Pseudo R-squ.:                  0.7475
Time:                        18:48:58   Log-Likelihood:                -56447.
converged:                       True   LL-Null:                   -2.2358e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      5.6153      0.079     71.261      0.000       5.461       5.770
Country_Austria            0.4503      0.229      1.963      0.050       0.001       0.900
Country_Belgium            0.6256      0.212      2.946      0.003       0.209       1.042
Country_Canada             0.0918      0.072      1.268      0.205      -0.050       0.234
Country_Denmark            0.6119      0.162      3.773      0.000       0.294       0.930
Country_France             0.6425      0.109      5.914      0.000       0.430       0.855
Country_Germany            0.0977      0.100      0.974      0.330      -0.099       0.294
Country_Hong Kong          1.6954      0.202      8.386      0.000       1.299       2.092
Country_Ireland           -0.0895      0.193     -0.463      0.643      -0.468       0.289
Country_Italy              0.2065      0.131      1.576      0.115      -0.050       0.463
Country_Mexico            -0.0593      0.139     -0.427      0.669      -0.331       0.213
Country_Netherlands        0.3691      0.114      3.243      0.001       0.146       0.592
Country_New Zealand        0.0504      0.142      0.355      0.722      -0.228       0.328
Country_Norway             0.5121      0.208      2.465      0.014       0.105       0.919
Country_Other              0.3602      0.464      0.776      0.438      -0.550       1.270
Country_Singapore         -0.2776      0.204     -1.360      0.174      -0.678       0.122
Country_Spain             -0.1249      0.134     -0.934      0.350      -0.387       0.137
Country_Sweden            -0.1137      0.134     -0.846      0.398      -0.377       0.150
Country_Switzerland        1.6459      0.199      8.265      0.000       1.256       2.036
Country_United Kingdom     0.2053      0.065      3.161      0.002       0.078       0.333
Country_United States      0.2358      0.060      3.916      0.000       0.118       0.354
Cat_Comics                -1.4078      0.050    -28.130      0.000      -1.506      -1.310
Cat_Crafts                -0.8171      0.059    -13.812      0.000      -0.933      -0.701
Cat_Dance                  1.0639      0.072     14.802      0.000       0.923       1.205
Cat_Design                -1.0409      0.038    -27.437      0.000      -1.115      -0.967
Cat_Fashion               -0.3164      0.042     -7.598      0.000      -0.398      -0.235
Cat_Film & Video           0.5709      0.032     18.111      0.000       0.509       0.633
Cat_Food                  -0.5259      0.040    -13.201      0.000      -0.604      -0.448
Cat_Games                 -2.2662      0.039    -58.088      0.000      -2.343      -2.190
Cat_Journalism            -0.3088      0.084     -3.678      0.000      -0.473      -0.144
Cat_Music                  0.3980      0.033     12.216      0.000       0.334       0.462
Cat_Photography           -0.1903      0.051     -3.735      0.000      -0.290      -0.090
Cat_Publishing            -0.4702      0.035    -13.286 